# One-Dimension Rotation

This is an example usage of the skydy package.

The base case is an object that can rotate about one axis.

It has one input torque, around the axis of rotation.

In [1]:
import skydy
from skydy.connectors import DOF, Connection, Joint
from skydy.multibody import MultiBody
from skydy.rigidbody import (
    Body,
    BodyCoordinate,
    BodyForce,
    BodyTorque,
    Ground,
    GroundCoordinate,
)
import matplotlib.pyplot as plt
import pprint


## Define the rotating Body

Name and create the body

In [2]:
# Give the car a name
rot_name = "1"

# Define the body
b_rot = Body(rot_name)


Give the Body some dimensions.

In [3]:
l_rot = 2
w_rot = 2
h_rot = 1

# Instantiate the car's dimensions
b_rot.dims.assign_values([l_rot, w_rot, h_rot])

In [4]:
# As it stands, this is simply a rigid body with 6 degrees of freedom and some geometry.
b_rot.positions()

Matrix([
[    x^1_G(t)],
[    y^1_G(t)],
[    z^1_G(t)],
[theta^1_x(t)],
[theta^1_y(t)],
[theta^1_z(t)]])

In [5]:
# Check the dimensions of the body.
b_rot.dims.as_dict()

{l^1_x: 2, l^1_y: 2, l^1_z: 1}

## Mass and Inertia Matrices

A Body comes equipped with a Mass and Inertia matrices. These are purely symbolic, but necessary to calculate the energy of the object. We do not physically calculate the moments of inertia, however, since we can define the Body in anyway, we align the body fixed axis with the principal inertial axes.

In [6]:
b_rot.mass_matrix.as_mat()

Matrix([
[m_1,   0,   0],
[  0, m_1,   0],
[  0,   0, m_1]])

In [7]:
b_rot.inertia_matrix.as_mat()

Matrix([
[I^1_xx, I^1_yx, I^1_zx],
[I^1_xy, I^1_yy, I^1_zy],
[I^1_xz, I^1_yz, I^1_zz]])

## External Force

Add a force at the Centre of Mass of the car.

In [8]:
# We can now add a torque to the body.
T1 = BodyTorque(name="1", z_dir=True)

# Force is applied at the COM
torque_loc = BodyCoordinate("PT1", 0, 0, 0)

# Add the torque at the location
b_rot.add_torque(T1, torque_loc)

## Define the Ground Body and Coordinate.

This acts as the reference point for our body, i.e., allows us to propagate relative positions into absolute.

In [9]:
b_gnd = Ground()
p_gnd = GroundCoordinate()

In [10]:
# Define the location of the body to the ground, in the body's body frame.
p_rot = BodyCoordinate("G1/O", 0, 0, 0)

## Degrees of Freedom

Define the coordinates that can move. We can also specify if a coordinate is fixed at a constant value. If the DOF is NOT specified, it is assumed to be fixed, with zero value.

In this case, let's fix the cart at height of 5m.

Note, index 0 represent the x-axis, 1 the y-, 2 the z-, 3 rotations about x-, 4 rotations about y- and 5 rotations about z-.

In [12]:
rot_dofs = [DOF(5)]

## Joint

This is the object that connects the two bodies. Eaach coordinate is in its respective body frame.

In [13]:
j1 = Joint(p_gnd, p_rot, rot_dofs, name=p_gnd.name)

## Connection

In [14]:
cnx_rot = Connection(b_gnd, j1, b_rot)

## MultiBody

In [15]:
oned_rot = MultiBody([cnx_rot,])

## Coordinates

In [16]:
oned_rot.coordinates

Matrix([[theta^1_z(t)]])

## Energy

In [17]:
oned_rot.kinetic_energy

0.5*I^1_zz*Derivative(theta^1_z(t), t)**2

In [18]:
oned_rot.potential_energy

0

## Kinetic Energy Metric

In [19]:
oned_rot.G

Matrix([[1.0*I^1_zz]])

## Equations of Motion

This returns the left and right hand sides of the equations of motion. The LHS will be linear in accelerations. The RHS is a function of the coordindates and inputs.

In [20]:
lhs, rhs = oned_rot.eoms()
lhs

Matrix([[1.0*I^1_zz*Derivative(theta^1_z(t), (t, 2))]])

In [21]:
rhs

Matrix([[tau^1_z]])

## Equilibria

Get the equilibria, either forced or unforced, for the described system.

In [22]:
oned_rot.get_equilibria()

([], [])

## System Representation

Generate the equations of the following form:

- Nonlinear: $G \dot{x} = f(x, u)$,
- Linear: $G \dot{x} = A x + B u$,

where $G$ is the Kinetic Energy Metric. Note, x represents the positions and velocities, A is the state-transition matrix and B is the control input matrix.

In [23]:
# Nonlinear
A, B = oned_rot.system_matrices(linearized=False)
A

Matrix([
[Derivative(theta^1_z(t), t)],
[                    tau^1_z]])

In [25]:
# Linear
A, B = oned_rot.system_matrices(linearized=True)
A

Matrix([
[0, 1],
[0, 0]])

In [26]:
B

Matrix([
[0],
[1]])

## Configuraton of bodies

In [27]:
pprint.pp(oned_rot.get_configuration())

{'0': {'coords': {x^0_G(t): 0.0,
                  y^0_G(t): 0.0,
                  z^0_G(t): 0.0,
                  theta^0_x(t): 0.0,
                  theta^0_y(t): 0.0,
                  theta^0_z(t): 0.0},
       'dims': {l^0_x: 0, l^0_y: 0, l^0_z: 0}},
 '1': {'coords': {x^1_G(t): 0.0,
                  y^1_G(t): 0.0,
                  z^1_G(t): 0.0,
                  theta^1_x(t): 0.0,
                  theta^1_y(t): 0.0,
                  theta^1_z(t): 1.0},
       'dims': {l^1_x: 2, l^1_y: 2, l^1_z: 1}}}
